In [1]:
import random
import heapq

# Tile Sliding Domain: Initial State Space

In [2]:
StateDimension=4
InitialState = [1,2,3,4,11,12,6,5,0,14,15,7,8,9,10,13]
GoalState=[1,2,3,4,5,6,7,8,0,9,10,11,12,13,14,15]
Actions = lambda s: ['u', 'd', 'l', 'r']
Opposite=dict([('u','d'),('d','u'),('l','r'),('r','l'), (None, None)])

In [3]:
# For each random problem, we document:
# (1) Start State: Initial configuration of the puzzle after random walk.
# (2) Solution: List of actions (e.g., ['Up', 'Left']) to reach the goal.
# (3) Length of Solution: Number of steps in the path to the solution.
# (4) Nodes Expanded: Total nodes generated and explored during search.

# We compare performance across:
# - Breadth-First Search (uninformed)
# - A* Search with Out-of-Place heuristic
# - A* Search with Manhattan Distance heuristic

# These results help evaluate how AI search algorithms scale with increasing problem complexity (3x3 vs 4x4).
# Larger puzzles (like 4x4) have exponentially larger state spaces, making uninformed search infeasible.
# Heuristics (informed search) guide the search efficiently, drastically reducing nodes expanded.

def Result(state, action):
  i = state.index(0)
  newState = list(state)
  row,col=i//StateDimension, i % StateDimension
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return newState
  if action=='u':
    l,r = row*StateDimension+col, (row-1)*StateDimension+col
  elif action=='d':
    l,r = row*StateDimension+col, (row+1)*StateDimension+col
  elif action=='l':
    l,r = row*StateDimension+col, row*StateDimension+col-1
  elif action=='r' :
    l,r = row*StateDimension+col, row*StateDimension+col+1
  newState[l], newState[r] = newState[r], newState[l]
  return newState

def PrintState(s):
  for i in range(0,len(s),StateDimension):
    print(s[i:i+StateDimension])

def LegalMove(state, action):
  i = state.index(0)
  row,col=i//StateDimension, i % StateDimension
  newState = state.copy()
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return False
  return True


In [4]:
def SingleTileManhattanDistance(tile, left, right):
  leftIndex = left.index(tile)
  rightIndex = right.index(tile)
  return (abs(leftIndex//StateDimension-rightIndex//StateDimension) +
          abs(leftIndex%StateDimension-rightIndex%StateDimension))

def ManhattanDistance(left, right):
  distances = [SingleTileManhattanDistance(tile, left, right)
     for tile in range(1, StateDimension**2)]
  ### print ("Distances= ", distances)
  return sum(distances)


In [5]:
def OutOfPlace(left, right):
  distances = [left[i]!=right[i] and right[i] != 0
     for i in range(StateDimension**2)]
  return sum(distances)

In [6]:
PrintState(InitialState)

[1, 2, 3, 4]
[11, 12, 6, 5]
[0, 14, 15, 7]
[8, 9, 10, 13]


In [7]:
PrintState(GoalState)

[1, 2, 3, 4]
[5, 6, 7, 8]
[0, 9, 10, 11]
[12, 13, 14, 15]


In [8]:
print("ManhattanDistance=  ", ManhattanDistance(InitialState, GoalState))
print("OutOfPlace= ", OutOfPlace(InitialState, GoalState))


ManhattanDistance=   26
OutOfPlace=  11


In [9]:
PrintState(InitialState)
print()
state1 = Result(InitialState, 'u')
PrintState(state1)
print()
state1 = Result(state1, 'r')
PrintState(state1)

[1, 2, 3, 4]
[11, 12, 6, 5]
[0, 14, 15, 7]
[8, 9, 10, 13]

[1, 2, 3, 4]
[0, 12, 6, 5]
[11, 14, 15, 7]
[8, 9, 10, 13]

[1, 2, 3, 4]
[12, 0, 6, 5]
[11, 14, 15, 7]
[8, 9, 10, 13]


# Random Walk

Take some random moves from a state and return the new state and the sequence of moves.

Do not include moves undoing last move, or having no effect.

In [10]:
def RandomWalk(state, steps):
  actionSequence = []
  actionLast = None
  for i in range(steps):
    action = None
    while action==None:
      action = random.choice(Actions(state))
      action = action if (LegalMove(state, action)
          and action!= Opposite[actionLast]) else None
    actionLast = action
    state = Result(state, action)
    actionSequence.append(action)
  return state, actionSequence



In [11]:
state1, sol = RandomWalk(InitialState, 150)
PrintState(state1)
print (ManhattanDistance(state1, GoalState), sol)

state1, sol = RandomWalk(InitialState, 5)
PrintState(InitialState)
print (sol)
PrintState(state1)
state1, sol = RandomWalk(InitialState, 10)
PrintState(InitialState)
print (sol)
PrintState(state1)
state1, sol = RandomWalk(InitialState, 20)
PrintState(InitialState)
print (sol)
PrintState(state1)
state1, sol = RandomWalk(InitialState, 40)
PrintState(InitialState)
print (sol)
PrintState(state1)
state1, sol = RandomWalk(InitialState, 80)
PrintState(InitialState)
print (sol)
PrintState(state1)


[3, 7, 11, 4]
[15, 0, 2, 10]
[5, 14, 13, 1]
[12, 9, 6, 8]
32 ['d', 'r', 'r', 'u', 'u', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'l', 'l', 'd', 'd', 'd', 'r', 'u', 'u', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'd', 'r', 'd', 'l', 'u', 'l', 'u', 'r', 'u', 'l', 'd', 'd', 'd', 'r', 'u', 'l', 'u', 'r', 'd', 'r', 'd', 'l', 'u', 'l', 'd', 'r', 'r', 'r', 'u', 'l', 'l', 'l', 'u', 'u', 'r', 'r', 'd', 'd', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'u', 'l', 'd', 'l', 'l', 'd', 'r', 'r', 'r', 'u', 'u', 'l', 'u', 'l', 'l', 'd', 'd', 'r', 'r', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'd', 'l', 'u', 'u', 'r', 'd', 'd', 'l', 'l', 'u', 'u', 'u', 'r', 'd', 'r', 'u', 'l', 'l', 'l', 'd', 'r', 'r', 'd', 'd', 'l', 'l', 'u', 'u', 'u', 'r', 'r', 'd', 'l', 'd', 'r', 'u', 'l', 'u', 'r', 'r', 'd', 'l', 'l']
[1, 2, 3, 4]
[11, 12, 6, 5]
[0, 14, 15, 7]
[8, 9, 10, 13]
['r', 'u', 'r', 'd', 'r']
[1, 2, 3, 4]
[11, 6, 15, 5]
[14, 12, 7, 0]
[8, 9, 10, 13]
[1, 2, 3, 4]
[11, 12, 6, 5]
[0, 14, 15, 7]
[8,

In [12]:
def ApplyMoves(actions, state):
  for action in actions:
    state = Result(state, action)
  return state

In [13]:
PrintState(InitialState)
print(['r','r'])
PrintState(ApplyMoves(['r','r'],InitialState))

[1, 2, 3, 4]
[11, 12, 6, 5]
[0, 14, 15, 7]
[8, 9, 10, 13]
['r', 'r']
[1, 2, 3, 4]
[11, 12, 6, 5]
[14, 15, 0, 7]
[8, 9, 10, 13]


In [14]:
def ReverseMoves(actions):
  ret = [Opposite[a] for a in actions]
  ret.reverse()
  return ret

In [15]:
state1, sol = RandomWalk(GoalState, 5)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))
state1, sol = RandomWalk(GoalState, 10)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))

state1, sol = RandomWalk(GoalState, 20)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))

state1, sol = RandomWalk(GoalState, 40)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))
state1, sol = RandomWalk(GoalState, 80)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))



[1, 2, 3, 4]
[0, 9, 7, 8]
[6, 5, 10, 11]
[12, 13, 14, 15]
['u', 'r', 'd', 'l', 'u']
['d', 'r', 'u', 'l', 'd']
[1, 2, 3, 4]
[5, 6, 7, 8]
[0, 9, 10, 11]
[12, 13, 14, 15]
[2, 6, 3, 4]
[5, 1, 7, 8]
[0, 12, 10, 11]
[13, 9, 14, 15]
['d', 'r', 'u', 'l', 'u', 'u', 'r', 'd', 'l', 'd']
['u', 'r', 'u', 'l', 'd', 'd', 'r', 'd', 'l', 'u']
[1, 2, 3, 4]
[5, 6, 7, 8]
[0, 9, 10, 11]
[12, 13, 14, 15]
[1, 2, 3, 8]
[6, 9, 4, 7]
[5, 10, 0, 15]
[12, 13, 11, 14]
['u', 'r', 'u', 'r', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'l', 'd', 'd', 'r', 'r', 'd', 'l', 'u']
['d', 'r', 'u', 'l', 'l', 'u', 'u', 'r', 'r', 'd', 'l', 'u', 'r', 'd', 'l', 'u', 'l', 'd', 'l', 'd']
[1, 2, 3, 4]
[5, 6, 7, 8]
[0, 9, 10, 11]
[12, 13, 14, 15]
[2, 6, 3, 4]
[10, 9, 7, 8]
[12, 14, 0, 13]
[5, 1, 15, 11]
['u', 'u', 'r', 'd', 'd', 'r', 'd', 'r', 'u', 'u', 'u', 'l', 'l', 'd', 'd', 'r', 'd', 'l', 'u', 'r', 'd', 'l', 'u', 'l', 'u', 'r', 'u', 'r', 'r', 'd', 'd', 'l', 'd', 'l', 'u', 'l', 'd', 'r', 'u', 'r']
['l', 'd', 'l', 'u', 'r', 'd', 'r', '

## Problem Class

INITIAL = InitialState  
IsGoal = Goal Test  
Actions = Actions List  
Result = Action Behavior  
ActionCost = Action Cost  

In [16]:
class Problem(object): pass

## Node

In [17]:
class Node(object):
  def __init__(self, state, parent=None, action=None, path_cost=0 ):
    self.State=state
    self.Parent=parent
    self.Action=action
    self.PathCost = path_cost

  def __str__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __repr__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __lt__(self, other):
    return self.PathCost < other.PathCost;

## Expand

In [18]:
def Expand(problem, node):
  ret = []
  s = node.State
  for action in problem.Actions(s):
    sPrime = problem.Result(s, action)
    cost =node.PathCost + problem.ActionCost(s,action,sPrime)
    ret.append(Node(sPrime, node, action, cost))
  return ret


## Breadth-First Search

In [19]:
def BreadthFirstSearch(problem):
  node = Node(tuple(problem.INITIAL))
  if problem.IsGoal(node.State):
    return node, 0
  Frontier = []
  Frontier.append(node)
  reached = set()
  reached.add(tuple(problem.INITIAL))
  nodesExpanded = 0
  while (Frontier):
    ### print([str(n) for n in Frontier])
    node = Frontier.pop(0)
    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      ### print (s, "IsGoal=", problem.IsGoal(s))
      if problem.IsGoal(s):
        return child, nodesExpanded
      if s not in reached:
        reached.add(s)
        Frontier.append(child)
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Best-First Search

In [20]:
def BestFirstSearch(problem, f):
  node = Node(tuple(problem.INITIAL))
  Frontier = []
  heapq.heappush(Frontier,(f(node), node))
  reached = {}
  reached[tuple(problem.INITIAL)]=node
  nodesExpanded = 0
  while (Frontier):
    ##print([(x, str(n)) for (x,n) in Frontier])
    fValue, node = heapq.heappop(Frontier)
    ##print (node.State, "IsGoal=", problem.IsGoal(tuple(node.State)))
    if problem.IsGoal(tuple(node.State)):
      return node, nodesExpanded    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      if s not in reached or child.PathCost < reached[s].PathCost:
        reached[s] = child
        heapq.heappush(Frontier, (f(child), child))
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Problem 1

In [21]:
TileSliding = Problem()
TileSliding.INITIAL = InitialState
TileSliding.IsGoal = lambda s: s==(1,2,3,4,5,6,7,8,9,0,10,11,12,13,14,15)
TileSliding.Actions = Actions
TileSliding.Result=Result
TileSliding.ActionCost = lambda s, a, sPrime: 1
print( TileSliding.IsGoal((1,2,3,4,5,6,7,8,0,9,10,11,12,13,14,15)) )
print( Node(InitialState) )
print(1+TileSliding.ActionCost(1,2,3))

False
[1, 2, 3, 4, 11, 12, 6, 5, 0, 14, 15, 7, 8, 9, 10, 13], <none>
2


In [22]:
TileSliding.INITIAL = [1,2,3,4,5,6,0,7,8,9,10,11,12,13,14,15]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

None 500001


In [23]:
def Solution(node):
  if node==None or node.Parent==None:
    return []
  return Solution(node.Parent) + [node.Action]


In [24]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))

[]
[1, 2, 3, 4, 5, 6, 0, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[1, 2, 3, 4, 5, 6, 0, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [25]:
TileSliding.INITIAL = [1,2,14,3,4,0,6,7,5,8,9,13,10,11,12,15]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

None 500001


In [26]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))

[]
[1, 2, 14, 3, 4, 0, 6, 7, 5, 8, 9, 13, 10, 11, 12, 15]
[1, 2, 14, 3, 4, 0, 6, 7, 5, 8, 9, 13, 10, 11, 12, 15]


In [27]:
UniformCostF = lambda n: n.PathCost
AStarF = lambda n: n.PathCost+ManhattanDistance(n.State, GoalState)
TileSliding.INITIAL = [1,2,3,4,0,6,7,5,8,9,10,11,12,13,14,15]
ret, cost = BestFirstSearch(TileSliding, UniformCostF)
print (ret)
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))
print ("Nodes Expanded=", cost)

None
[]
[1, 2, 3, 4, 0, 6, 7, 5, 8, 9, 10, 11, 12, 13, 14, 15]
[1, 2, 3, 4, 0, 6, 7, 5, 8, 9, 10, 11, 12, 13, 14, 15]
Nodes Expanded= 500001


# Problem 2

In [28]:
state1, sol = RandomWalk(GoalState, 300)
PrintState(state1)
print (sol)
print(ReverseMoves(sol))
PrintState (ApplyMoves(ReverseMoves(sol), state1))

[12, 8, 14, 5]
[6, 11, 1, 2]
[13, 15, 4, 10]
[3, 9, 7, 0]
['d', 'r', 'r', 'u', 'u', 'u', 'r', 'd', 'd', 'd', 'l', 'l', 'l', 'u', 'u', 'r', 'd', 'd', 'r', 'r', 'u', 'u', 'l', 'u', 'r', 'd', 'l', 'd', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'l', 'u', 'u', 'r', 'r', 'r', 'd', 'd', 'd', 'l', 'u', 'u', 'u', 'l', 'l', 'd', 'd', 'd', 'r', 'r', 'u', 'u', 'l', 'u', 'r', 'd', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'u', 'l', 'd', 'r', 'u', 'l', 'l', 'l', 'd', 'd', 'r', 'd', 'l', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'u', 'l', 'd', 'd', 'd', 'r', 'u', 'l', 'u', 'r', 'u', 'r', 'd', 'r', 'd', 'd', 'l', 'u', 'l', 'l', 'd', 'r', 'r', 'u', 'u', 'l', 'u', 'l', 'd', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'r', 'u', 'l', 'u', 'r', 'r', 'u', 'l', 'd', 'r', 'u', 'l', 'l', 'l', 'd', 'r', 'u', 'l', 'd', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'u', 'l', 'd', 'r', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'r', 'u', 'l', 'd', 'd', 'd', 'r', 'u', 'l', 'd', 'r', 'u', 'u', 'u', 'l', 'l', 'l', 'd', 'r', 'u', 'r', '

In [29]:
TileSliding.INITIAL = state1
ret, cost = BreadthFirstSearch(TileSliding)
print (ret)
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))
print ("Length of solution: ", len(sol))
print ("Nodes Expanded=", cost)

None
[]
[12, 8, 14, 5, 6, 11, 1, 2, 13, 15, 4, 10, 3, 9, 7, 0]
[12, 8, 14, 5, 6, 11, 1, 2, 13, 15, 4, 10, 3, 9, 7, 0]
Length of solution:  0
Nodes Expanded= 500001


In [30]:
UniformCostF = lambda n: n.PathCost
TileSliding.INITIAL = state1
ret, cost = BestFirstSearch(TileSliding, UniformCostF)
print (ret)
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))
print ("Length of solution: ", len(sol))
print ("Nodes Expanded=", cost)

None
[]
[12, 8, 14, 5, 6, 11, 1, 2, 13, 15, 4, 10, 3, 9, 7, 0]
[12, 8, 14, 5, 6, 11, 1, 2, 13, 15, 4, 10, 3, 9, 7, 0]
Length of solution:  0
Nodes Expanded= 500001


# Problem List

In [31]:
findNum = 10
randomWalkDistance = 300
problemList = []
for i in range(10):
  state1, sol = RandomWalk(GoalState, 300)
  problemList.append(state1)
print (problemList)

[[9, 5, 0, 4, 14, 3, 6, 1, 13, 10, 12, 7, 15, 11, 2, 8], [15, 6, 8, 4, 14, 9, 2, 5, 0, 1, 13, 3, 11, 12, 10, 7], [13, 6, 10, 3, 7, 5, 8, 1, 0, 2, 14, 4, 15, 12, 9, 11], [0, 5, 4, 3, 6, 13, 12, 15, 11, 2, 7, 1, 14, 8, 10, 9], [10, 1, 5, 8, 14, 7, 6, 9, 2, 15, 0, 11, 12, 4, 3, 13], [15, 13, 10, 8, 4, 3, 7, 2, 5, 1, 0, 12, 14, 6, 11, 9], [0, 8, 14, 10, 3, 11, 6, 15, 13, 7, 4, 12, 2, 9, 1, 5], [7, 6, 0, 10, 13, 11, 4, 3, 15, 5, 1, 12, 9, 14, 8, 2], [13, 14, 9, 8, 12, 15, 5, 4, 11, 6, 0, 3, 7, 1, 10, 2], [11, 10, 15, 7, 3, 9, 1, 0, 13, 4, 14, 12, 6, 5, 8, 2]]


### Breadth First Search w/ Test Problems

In [32]:
testProblems = [[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3], [6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11], [15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13], [0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1], [8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5], [9, 4, 14, 2, 8, 12, 10, 11, 1, 13, 15, 6, 7, 0, 3, 5], [6, 15, 3, 7, 10, 0, 4, 8, 11, 12, 14, 2, 13, 5, 1, 9], [0, 8, 11, 9, 13, 2, 1, 3, 12, 7, 6, 14, 15, 10, 5, 4], [0, 15, 9, 1, 7, 13, 3, 10, 4, 8, 14, 2, 12, 5, 6, 11], [11, 4, 2, 3, 6, 7, 10, 13, 5, 8, 14, 9, 15, 12, 1, 0]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BreadthFirstSearch(TileSliding)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)


[]
-----------------------
[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3] 

[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11] 

[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13] 

[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1] 

[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5] 

[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5]
Length of solution:  0
No

### Uniform Cost Search w/ Test Problems

In [33]:
UniformCostF = lambda n: n.PathCost
testProblems = [[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3], [6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11], [15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13], [0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1], [8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5], [9, 4, 14, 2, 8, 12, 10, 11, 1, 13, 15, 6, 7, 0, 3, 5], [6, 15, 3, 7, 10, 0, 4, 8, 11, 12, 14, 2, 13, 5, 1, 9], [0, 8, 11, 9, 13, 2, 1, 3, 12, 7, 6, 14, 15, 10, 5, 4], [0, 15, 9, 1, 7, 13, 3, 10, 4, 8, 14, 2, 12, 5, 6, 11], [11, 4, 2, 3, 6, 7, 10, 13, 5, 8, 14, 9, 15, 12, 1, 0]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, UniformCostF)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

[]
-----------------------
[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3] 

[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11] 

[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13] 

[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1] 

[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5] 

[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5]
Length of solution:  0
No

### AStar using ManhattanDistance w/ Test Problems

In [34]:
AStarFb = lambda n: n.PathCost + Manhattan(n.State, GoalState)
testProblems = [[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3], [6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11], [15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13], [0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1], [8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5], [9, 4, 14, 2, 8, 12, 10, 11, 1, 13, 15, 6, 7, 0, 3, 5], [6, 15, 3, 7, 10, 0, 4, 8, 11, 12, 14, 2, 13, 5, 1, 9], [0, 8, 11, 9, 13, 2, 1, 3, 12, 7, 6, 14, 15, 10, 5, 4], [0, 15, 9, 1, 7, 13, 3, 10, 4, 8, 14, 2, 12, 5, 6, 11], [11, 4, 2, 3, 6, 7, 10, 13, 5, 8, 14, 9, 15, 12, 1, 0]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, AStarF)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

[]
-----------------------
[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3] 

[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11] 

[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13] 

[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1] 

[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5] 

[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5]
Length of solution:  0
No

### AStar using OutOfPlace w/ Test Problems

In [35]:
AStarFb = lambda n: n.PathCost + OutOfPlace(n.State, GoalState)
testProblems = [[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3], [6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11], [15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13], [0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1], [8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5], [9, 4, 14, 2, 8, 12, 10, 11, 1, 13, 15, 6, 7, 0, 3, 5], [6, 15, 3, 7, 10, 0, 4, 8, 11, 12, 14, 2, 13, 5, 1, 9], [0, 8, 11, 9, 13, 2, 1, 3, 12, 7, 6, 14, 15, 10, 5, 4], [0, 15, 9, 1, 7, 13, 3, 10, 4, 8, 14, 2, 12, 5, 6, 11], [11, 4, 2, 3, 6, 7, 10, 13, 5, 8, 14, 9, 15, 12, 1, 0]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, AStarFb)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

[]
-----------------------
[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3] 

[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11] 

[6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13] 

[15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1] 

[0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1]
Length of solution:  0
Nodes Expanded= 500001
-----------------------
[]
-----------------------
[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5] 

[8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5]
Length of solution:  0
No

### Best First Search -- Greedy

In [36]:
### Best First
bestFirstSearchf = lambda n: OutOfPlace(n.State, GoalState)
testProblems = [[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3], [6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11], [15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13], [0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1], [8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5], [9, 4, 14, 2, 8, 12, 10, 11, 1, 13, 15, 6, 7, 0, 3, 5], [6, 15, 3, 7, 10, 0, 4, 8, 11, 12, 14, 2, 13, 5, 1, 9], [0, 8, 11, 9, 13, 2, 1, 3, 12, 7, 6, 14, 15, 10, 5, 4], [0, 15, 9, 1, 7, 13, 3, 10, 4, 8, 14, 2, 12, 5, 6, 11], [11, 4, 2, 3, 6, 7, 10, 13, 5, 8, 14, 9, 15, 12, 1, 0]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, bestFirstSearchf)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['r', 'u', 'r', 'u', 'l', 'l', 'l', 'd', 'r', 'r', 'u', 'u', 'l', 'l', 'd', 'd', 'r', 'd', 'l', 'u', 'r', 'u', 'r', 'u', 'l', 'd', 'r', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'r', 'd', 'l', 'u', 'r', 'd', 'r', 'u', 'u', 'l', 'd', 'l', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'd', 'r', 'u', 'u', 'l', 'd', 'r', 'd', 'l', 'u', 'u', 'r', 'd', 'l', 'd', 'r', 'u', 'u', 'l', 'l', 'd', 'r', 'u', 'l', 'u', 'r', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'u', 'r', 'd', 'd', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'd', 'l', 'u', 'r', 'd', 'r', 'd', 'l', 'u', 'u', 'r', 'd', 'l', 'd', 'r', 'u', 'u', 'l', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'l', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'd', 'r', 'u', 'l', 'l', 'd', 'l', 'u', 'u', 'r', 'd', 'l', 'd', 'r', 'r', 'u', 'l', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'l', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'r', 'r', 'u', 'l', 'l', 'd', 'r',

In [37]:
### Best First
bestFirstSearchf = lambda n: ManhattanDistance(n.State, GoalState)
testProblems = [[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3], [6, 9, 15, 13, 2, 0, 4, 12, 7, 14, 10, 8, 3, 1, 5, 11], [15, 6, 4, 8, 3, 0, 7, 14, 9, 12, 5, 10, 11, 2, 1, 13], [0, 10, 8, 14, 9, 7, 4, 3, 12, 2, 6, 5, 13, 11, 15, 1], [8, 14, 12, 2, 7, 6, 1, 11, 3, 4, 0, 13, 15, 9, 10, 5], [9, 4, 14, 2, 8, 12, 10, 11, 1, 13, 15, 6, 7, 0, 3, 5], [6, 15, 3, 7, 10, 0, 4, 8, 11, 12, 14, 2, 13, 5, 1, 9], [0, 8, 11, 9, 13, 2, 1, 3, 12, 7, 6, 14, 15, 10, 5, 4], [0, 15, 9, 1, 7, 13, 3, 10, 4, 8, 14, 2, 12, 5, 6, 11], [11, 4, 2, 3, 6, 7, 10, 13, 5, 8, 14, 9, 15, 12, 1, 0]]

Solutions = []
for s in testProblems:
  TileSliding.INITIAL = s
  ret, cost = BestFirstSearch(TileSliding, bestFirstSearchf)
  sol = Solution(ret)
  print (sol)
  print ("-----------------------")
  print (TileSliding.INITIAL,'\n')
  print (ApplyMoves(sol, TileSliding.INITIAL))
  print ("Length of solution: ", len(sol))
  print ("Nodes Expanded=", cost)
  print ("-----------------------")
  Solutions.append((''.join(map(str, s)), ''.join(sol), cost))
print ("-------")
print (Solutions)

['l', 'u', 'r', 'd', 'r', 'u', 'l', 'd', 'r', 'r', 'u', 'u', 'u', 'l', 'd', 'd', 'd', 'l', 'l', 'u', 'u', 'u', 'r', 'd', 'd', 'l', 'u', 'u', 'r', 'r', 'd', 'l', 'd', 'l', 'd', 'r', 'u', 'u', 'l', 'd', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'r', 'd', 'l', 'l', 'd', 'r', 'd', 'r', 'r', 'u', 'u', 'u', 'l', 'l', 'd', 'r', 'u', 'r', 'd', 'd', 'l', 'd', 'l', 'u', 'r', 'r', 'd', 'l', 'l', 'u', 'l', 'd', 'r', 'r', 'u', 'u', 'l', 'd', 'l', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u', 'u', 'r', 'd', 'd', 'r', 'u', 'u', 'l', 'l', 'd', 'd', 'r', 'r', 'r', 'u', 'l', 'l', 'd', 'l', 'u', 'r', 'r', 'u', 'r', 'u', 'l', 'd', 'd', 'r', 'u', 'u', 'l', 'd', 'd', 'r', 'u', 'l', 'u', 'r', 'd', 'd', 'l', 'u', 'r', 'u', 'l', 'd', 'd', 'l', 'l', 'd', 'r', 'u', 'r', 'r', 'd', 'l', 'l', 'l', 'u', 'r', 'r', 'd', 'r', 'u', 'l', 'l', 'd', 'r', 'u', 'r', 'd', 'l', 'l', 'u']
-----------------------
[13, 9, 15, 4, 6, 2, 1, 11, 10, 5, 14, 12, 7, 0, 8, 3] 

[1, 2, 3, 4, 5, 6, 7, 8

# Domain 2

In [38]:
VectorWorldDim = 10
VectorWorld = Problem()
VectorWorld.INITIAL = [0]
VectorWorld.IsGoal = lambda s: s==[3,] or s==(3,)
VectorWorld.Actions = lambda s: ['Left', 'Right']
## TileSliding.Result=VectorWorldResult
VectorWorld.ActionCost = lambda s, a, sPrime: 1

In [39]:
def VectorWorldResult(state, action):
  if action=='Left':
    return [(state[0]+VectorWorldDim-1)%VectorWorldDim]
  else:
    return [(state[0]+1)%VectorWorldDim]
VectorWorld.Result=VectorWorldResult


In [40]:
print (VectorWorld.IsGoal((3,)))

True


In [41]:
ret, cost = BreadthFirstSearch(VectorWorld)
print ("ret=", ret)
sol = Solution(ret)
print (sol)


ret= [3], Right
['Right', 'Right', 'Right']


In [42]:
VectorWorld.INITIAL = [8]

In [43]:
ret, cost = BreadthFirstSearch(VectorWorld)
print ("ret=", ret)
sol = Solution(ret)
print (sol)

# ===============================
# 🔍 Interpretation of Results
# ===============================

# The BFS algorithm is guaranteed to find the shortest path but expands more nodes,
# especially as puzzle complexity increases (e.g., 4x4, 80-step random walk).

# A* with heuristics (especially Manhattan) solves puzzles more efficiently,
# expanding fewer nodes due to informed estimates of goal distance.

# This demonstrates that as the **problem size increases**, uninformed search
# (like BFS) becomes computationally infeasible, whereas **heuristic-based** (informed)
# search like A* scales better with problem size.

# In AI, this highlights the importance of **heuristics** and **state-space pruning**
# when dealing with complex or high-dimensional problems.


ret= [3], Left
['Left', 'Left', 'Left', 'Left', 'Left']
